<a href="https://colab.research.google.com/github/Daprosero/Procesamiento_Lenguaje_Natural/blob/main/2.%20Transformer/Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit -q # El comando pip install streamlit -q se utiliza para instalar el paquete Streamlit ya que no viene por defecto en python
# Instalar Streamlit
!pip install streamlit

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared


--2025-10-29 04:30:11--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.10.0/cloudflared-linux-amd64 [following]
--2025-10-29 04:30:11--  https://github.com/cloudflare/cloudflared/releases/download/2025.10.0/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/3cf4ee00-005b-4d19-8b1e-15145bd129bf?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-10-29T05%3A12%3A53Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-10-29

In [ ]:
%%writefile primer_streamlit.py
import streamlit as st

# Título de la app
st.title("Mi primera aplicación con Streamlit")

# # Texto introductorio
st.write("¡Bienvenidos a mi aplicación interactiva!")

# # Agregar un botón
if st.button("Presiona aquí"):
    st.write("¡El botón ha sido presionado!")

# Agregar un área de entrada de texto
user_input = st.text_input("Ingresa tu nombre y presiona enter:", "")
if user_input:
    st.write(f"¡Hola, {user_input}!")

# Agregar una sección para una calculadora simple
st.write("### Calculadora simple")

# Entradas para los números
num1 = st.number_input("Ingresa el primer número", value=0)
num2 = st.number_input("Ingresa el segundo número", value=0)

# Botón para sumar los números
if st.button("Sumar"):
    resultado = num1 + num2
    st.write(f"El resultado de la suma es: {resultado}")

# Entradas para los números
num3 = st.number_input("Ingresa el primer número", value=0, key='tercero')
num4 = st.number_input("Ingresa el segundo número", value=0, key='cuarto')

# Botón para sumar los números
if st.button("Multiplicar"):
    resultado = num3 * num4
    st.write(f"El resultado de la multiplicación es: {resultado}")

# Texto final
st.write("Puedes incluir cualquier contenido en esta aplicación, incluyendo cálculos, visualizaciones, e interacción con el usuario.")

Overwriting primer_streamlit.py


In [ ]:
def run():
    # Ejecutar Streamlit
    !streamlit run primer_streamlit.py &>/content/logs.txt &

    # Exponer el puerto 8501 con Cloudflare Tunnel
    !cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

    # Leer la URL pública generada por Cloudflare usando '|' como guía (con fallback)
    import time, re

    time.sleep(5)  # breve espera

    url = None
    with open('/content/cloudflared.log', encoding='utf-8', errors='ignore') as f:
        for line in f:
            # 1) Intento principal: tomar el texto entre barras verticales
            if 'trycloudflare.com' in line and '|' in line:
                # Divide por '|' y busca el fragmento que empiece con http
                candidates = [p.strip() for p in line.split('|') if p.strip()]
                for c in candidates:
                    if c.startswith('http') and 'trycloudflare.com' in c:
                        url = c
                        break
                if url:
                    break

            # 2) Respaldo: regex por si el formato cambia
            if 'trycloudflare.com' in line and url is None:
                m = re.search(r"https?://[^\s]*trycloudflare\.com[^\s]*", line)
                if m:
                    url = m.group(0).strip(' |')
                    break

    if url:
        print(f"Tu aplicación está disponible en: {url}")
    else:
        print("No encontré la URL aún. Prueba aumentar el time.sleep o revisa el log con:")
        print("!tail -n 40 /content/cloudflared.log")




In [ ]:
run()

Tu aplicación está disponible en: https://pastor-budget-periodic-quite.trycloudflare.com


In [ ]:
def kill():
    !pkill streamlit

In [ ]:
kill()

In [ ]:
%%writefile primer_streamlit.py
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import io

# ========== Configuración ==========
st.set_page_config(page_title="Mini-EDA", page_icon="📊", layout="wide")
st.title("📊 Mini-EDA — Vista por Bloques desde Sidebar")

# ========== Datos base (compartidos) ==========
@st.cache_data
def datos_ejemplo(n=300, seed=42):
    np.random.seed(seed)
    return pd.DataFrame({
        "categoria": np.random.choice(["A","B","C"], size=n),
        "valor_1": np.random.normal(50, 10, size=n),
        "valor_2": np.random.uniform(0, 1, size=n),
        "fecha": pd.date_range("2024-01-01", periods=n, freq="D"),
    })

# Control global (tamaño muestra)
st.sidebar.header("Configuración global")
tam = st.sidebar.slider("Tamaño de la muestra", 50, 2000, 300, 50, key="tam_global")
df_base = datos_ejemplo(tam)

# ========== Menú lateral (router) ==========
st.sidebar.header("🧭 Navegación")
bloque = st.sidebar.radio(
    "Selecciona un bloque:",
    [
        "Datos & KPIs",
        "Filtros y Tabla",
        "Gráficas básicas",
        "Matriz de correlación",
        "Pestañas + Descargas",
        "Serie temporal",
    ],
    index=0,
)

# ========== BLOQUE 1: Datos & KPIs ==========
if bloque == "Datos & KPIs":
    st.subheader("📄 Datos & KPIs")
    st.dataframe(df_base.head(30), use_container_width=True)
    c1, c2, c3 = st.columns(3)
    c1.metric("Filas", f"{len(df_base):,}")
    c2.metric("Media valor_1", f"{df_base['valor_1'].mean():.2f}")
    c3.metric("Nulos", f"{df_base.isna().sum().sum():,}")

# ========== BLOQUE 2: Filtros y Tabla ==========
elif bloque == "Filtros y Tabla":
    st.subheader("🧹 Filtros y Tabla")
    st.sidebar.caption("Controles del bloque: Filtros y Tabla")
    col_num = st.sidebar.selectbox("Columna numérica", ["valor_1", "valor_2"], key="num_filtro")
    vmin, vmax = float(df_base[col_num].min()), float(df_base[col_num].max())
    rnum = st.sidebar.slider(f"Rango para {col_num}", vmin, vmax, (vmin, vmax), key="rango_num")
    cats = sorted(df_base["categoria"].unique())
    sel = st.sidebar.multiselect("Categorías", cats, default=cats, key="cats_filtro")

    df_f = df_base[df_base[col_num].between(rnum[0], rnum[1])]
    df_f = df_f[df_f["categoria"].isin(sel)]

    st.dataframe(df_f.head(50), use_container_width=True)
    st.caption(f"Coincidencias: {len(df_f):,}")

# ========== BLOQUE 3: Gráficas básicas ==========
elif bloque == "Gráficas básicas":
    st.subheader("📈 Gráficas básicas")
    st.sidebar.caption("Controles del bloque: Gráficas básicas")
    num_cols = ["valor_1", "valor_2"]
    col_x = st.sidebar.selectbox("Eje X (numérico)", num_cols, index=0, key="x_graf")
    col_y = st.sidebar.selectbox("Eje Y (numérico)", num_cols, index=1, key="y_graf")
    color = st.sidebar.selectbox("Color (categoría)", [None, "categoria"], index=1, key="color_graf")

    c1, c2 = st.columns(2)
    with c1:
        st.write("Histograma")
        st.plotly_chart(px.histogram(df_base, x=col_x, nbins=30, color=color), use_container_width=True)
    with c2:
        st.write("Dispersión")
        st.plotly_chart(px.scatter(df_base, x=col_x, y=col_y, color=color, trendline="ols"), use_container_width=True)

# ========== BLOQUE 4: Matriz de correlación ==========
elif bloque == "Matriz de correlación":
    st.subheader("🔥 Matriz de correlación")
    num_cols = [c for c in df_base.columns if pd.api.types.is_numeric_dtype(df_base[c])]
    if len(num_cols) >= 2:
        corr = df_base[num_cols].corr(numeric_only=True)
        st.plotly_chart(px.imshow(corr, text_auto=True, aspect="auto"), use_container_width=True)
    else:
        st.info("Se requieren ≥ 2 columnas numéricas.")

# ========== BLOQUE 5: Pestañas + Descargas ==========
elif bloque == "Pestañas + Descargas":
    st.subheader("🗂️ Pestañas + Descargas")
    st.sidebar.caption("Controles del bloque: Pestañas + Descargas")
    col_num = st.sidebar.selectbox("Numérica", ["valor_1", "valor_2"], key="num_tabs")
    vmin, vmax = float(df_base[col_num].min()), float(df_base[col_num].max())
    rnum = st.sidebar.slider("Rango", vmin, vmax, (vmin, vmax), key="rango_tabs")
    cats = sorted(df_base["categoria"].unique())
    sel = st.sidebar.multiselect("Categorías", cats, default=cats, key="cats_tabs")

    df_f = df_base[df_base[col_num].between(rnum[0], rnum[1])]
    df_f = df_f[df_f["categoria"].isin(sel)]

    tab1, tab2, tab3 = st.tabs(["📄 Resumen", "📈 Gráficas", "⬇️ Descargas"])

    with tab1:
        st.write("Resumen de datos (filtrado)")
        st.dataframe(df_f.head(50), use_container_width=True)
        c1, c2, c3 = st.columns(3)
        c1.metric("Filas filtradas", len(df_f))
        c2.metric("Media valor_1", f"{df_f['valor_1'].mean():.2f}")
        c3.metric("Media valor_2", f"{df_f['valor_2'].mean():.2f}")

    with tab2:
        st.write("Visualizaciones")
        st.plotly_chart(px.histogram(df_f, x="valor_1", nbins=30, color="categoria"), use_container_width=True)
        st.plotly_chart(px.scatter(df_f, x="valor_1", y="valor_2", color="categoria", trendline="ols"), use_container_width=True)

    with tab3:
        st.write("Exportar CSV/PNG")
        csv_bytes = df_f.to_csv(index=False).encode("utf-8")
        st.download_button("Descargar CSV filtrado", data=csv_bytes, file_name="filtrado.csv", mime="text/csv")

        fig = px.scatter(df_f, x="valor_1", y="valor_2", color="categoria", title="Dispersión filtrada")
        st.plotly_chart(fig, use_container_width=True)
        buf = io.BytesIO()
        fig.write_image(buf, format="png")
        st.download_button("Descargar figura (PNG)", data=buf.getvalue(), file_name="grafico.png", mime="image/png")

# ========== BLOQUE 6: Serie temporal ==========
elif bloque == "Serie temporal":
    st.subheader("🕒 Serie temporal")
    st.sidebar.caption("Controles del bloque: Serie temporal")
    # Filtro de fechas local del bloque
    fecha_min, fecha_max = df_base["fecha"].min(), df_base["fecha"].max()
    rango = st.date_input("Rango de fechas", (fecha_min, fecha_max), key="fecha_st")
    df = df_base.copy()
    if isinstance(rango, tuple) and len(rango) == 2:
        df = df[(df["fecha"] >= pd.to_datetime(rango[0])) & (df["fecha"] <= pd.to_datetime(rango[1]))]

    st.write("Promedio diario de valor_1")
    serie = df.groupby("fecha")["valor_1"].mean()
    st.line_chart(serie)


Overwriting primer_streamlit.py


In [ ]:
run()

Tu aplicación está disponible en: https://sea-stunning-hybrid-hist.trycloudflare.com
